# Compare local QARTOD Climatology and Gross Range Test Results to Expected Flags

### Import modules used in this notebook

In [ ]:
# Import libraries
import os
import re
import gc
import io
import ast
import pandas as pd
import numpy as np
import xarray as xr
import warnings
warnings.filterwarnings("ignore")
import sys

In [ ]:
# Import OOINet library
from ooinet import M2M
from ooinet.Instrument.common import process_file

In [ ]:
# Import functions from ooi-data-explorations library
from ooi_data_explorations.uncabled.process_dosta import dosta_datalogger
from ooi_data_explorations.combine_data import combine_datasets

In [ ]:
# Import dask tools and ProgressBar
import dask
from dask.diagnostics import ProgressBar

### Define reference designator and file paths for chosen variable

In [ ]:
# Set reference designator, data stream, and method 

site = "CP01CNSM"                                   # Coastal Pioneer Array (NES) - Central Surface Mooring
node = "MFD37"                                      # 
sensor = "03-CTDBPD000"                             # CTD Bottom-pumped
method = "recovered_inst"                           # non-decimated data from recovered instrument
stream = "ctdbp_cdef_instrument_recovered"          # name of data stream

refdes = '-'.join((site,node,sensor))               # build reference designator

type = 'prod'                                       # dataset saved from OOINet/"production" or from dev1

In [ ]:
# Build path to data files with local test results and load to workspace
from qartod_testing.data_processing import build_data_path
ds_path = build_data_path(refdes,method,stream,'prod',folder='interim')

# Load data from .nc files
ds_expected = xr.open_dataset(ds_path)
ds_expected

### Load test QARTOD test flags from local processed dataset

In [ ]:
# Build path to data files with local test results and load to workspace
from qartod_testing.data_processing import build_data_path
ds_path = build_data_path(refdes,method,stream,'prod',folder='processed')

# Load data from .nc files
ds_local = xr.open_dataset(ds_path)
ds_local

### Extract expected QC results from local interim dataset

In [ ]:
# Create a dictionary of key-value pairs of dataset variable name:alternate parameter name
test_parameters={}
for var in ds_prod.variables:
    if "qartod_results" in var:
        # Get the parameter name
        param = var.split("_qartod")[0]
        
        # Check if the parameter has an alternative ooinet_name
        if "alternate_parameter_name" in ds_prod[param].attrs:
            ooinet_name = ds_prod[param].attrs["alternate_parameter_name"]
        else:
            ooinet_name = param
        
        # Save the results in a dictionary
        test_parameters.update({
            param: ooinet_name
        })
# Print out the results
test_parameters

In [ ]:
results_expected = ds_expected[test_parameters]

### Comparing local results of QARTOD tests to expected results 

In [ ]:
comparison = []
for param in test_parameters:
    comparison[param] = results_expected[param] == ds_local[param]

In [ ]:
# or something like Andrew's example:

def run_comparison(ds, param, test_results):
    """
    Runs a comparison between the qartod results returned as part of the dataset
    and results calculated locally.
    """
    # Get the local test results and convert to string type for comparison
    local_results = test_results[param].astype(str)
    
    # Run comparison
    not_equal = np.where(ds[f"{param}_qartod_gross_range_test"] != local_results)[0]
    
    if len(not_equal) == 0:
        return None
    else:
        return not_equal

### Identify differences in results

### Export differing values to CSV with relevant metadata
metadata could include: date & time, sensor, node, site, local & expected test results... whatever